In this notebook we stage some data in GCS buckets for cloud-based training. We could do all of this in the main notebook but it's not too relevant for the demo itself, so we are breaking it out and only running it once.

# Setup GCP environment

* Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos
* Cloud training often involves saving and restoring model files. Therefore, we should <b>create a single-region bucket</b>. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available)

<b>Change the cell below</b> to reflect your Project ID and bucket name.

In [1]:
%%bash
source activate py2env
conda install -y pytz
pip uninstall -y google-cloud-dataflow
pip install --upgrade apache-beam[gcp]

Solving environment: ...working... done

# All requested packages already installed.

Requirement already up-to-date: apache-beam[gcp] in /usr/local/envs/py2env/lib/python2.7/site-packages (2.7.0)




==> WARNING: A newer version of conda exists. <==
  current version: 4.5.10
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base -c defaults conda


Skipping google-cloud-dataflow as it is not installed.


In [2]:
PROJECT = 'rostlab-181304'    # CHANGE THIS
BUCKET = 'rostlab-181304-ml' # REPLACE WITH YOUR BUCKET NAME. Use a regional bucket in the region you selected.
REGION = 'us-central1' # Choose an available region for Cloud MLE from https://cloud.google.com/ml-engine/docs/regions.

In [5]:
import datalab.bigquery as bq
import pandas as pd
import os
import shutil
import apache_beam as beam
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8' 

## ensure we're using python2 env
os.environ['CLOUDSDK_PYTHON'] = 'python2'

In [6]:
%%bash
## ensure gcloud is up to date
gcloud components update

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`


All components are up to date.
Updated property [core/project].
Updated property [compute/region].
Updated property [ml_engine/local_python].


## Specifying query to pull the data

Let's pull out a few extra columns from the timestamp.

In [7]:
def create_query(phase, EVERY_N):
  if EVERY_N == None:
    EVERY_N = 4 #use full dataset
    
  #select and pre-process fields
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  DAYOFWEEK(pickup_datetime) AS dayofweek,
  HOUR(pickup_datetime) AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """
  
  #add subsampling criteria by modding with hashkey
  if phase == 'train': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} < 2".format(base_query,EVERY_N)
  elif phase == 'valid': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 2".format(base_query,EVERY_N)
  elif phase == 'test':
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 3".format(base_query,EVERY_N)
  return query
    
print(create_query('valid', 100)) #example query using 1% of data


SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  DAYOFWEEK(pickup_datetime) AS dayofweek,
  HOUR(pickup_datetime) AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
   AND ABS(HASH(pickup_datetime)) % 100 == 2


Preprocessing Dataflow job from BigQuery This code reads from BigQuery and saves the data as-is on Google Cloud Storage. We can do additional preprocessing and cleanup inside Dataflow, but then we'll have to remember to repeat that prepreprocessing during inference. It is better to use tf.transform which will do this book-keeping for you, or to do preprocessing within your TensorFlow model. We will look at this in future notebooks. For now, we are simply moving data from BigQuery to CSV using Dataflow.
While we could read from BQ directly from TensorFlow (See: https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), it is quite convenient to export to CSV and do the training off CSV. Let's use Dataflow to do this at scale.

Because we are running this on the Cloud, you should go to the GCP Console (https://console.cloud.google.com/dataflow) to look at the status of the job. It will take several minutes for the preprocessing job to launch.

In [8]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/taxi_preproc/
fi

In [17]:
import datetime

####
# Arguments:
#   -rowdict: Dictionary. The beam bigquery reader returns a PCollection in
#     which each row is represented as a python dictionary
# Returns:
#   -rowstring: a comma separated string representation of the record with dayofweek
#     converted from int to string (e.g. 3 --> Tue)
####
def to_csv(rowdict):
  days = ['null', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
  CSV_COLUMNS = 'fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key'.split(',')
  rowdict['dayofweek'] = days[rowdict['dayofweek']]
  rowstring = ','.join([str(rowdict[k]) for k in CSV_COLUMNS])
  return rowstring


####
# Arguments:
#   -EVERY_N: Integer. Sample one out of every N rows from the full dataset.
#     Larger values will yield smaller sample
#   -RUNNER: 'DirectRunner' or 'DataflowRunner'. Specfy to run the pipeline
#     locally or on Google Cloud respectively. 
# Side-effects:
#   -Creates and executes dataflow pipeline. 
#     See https://beam.apache.org/documentation/programming-guide/#creating-a-pipeline
####
def preprocess(EVERY_N, RUNNER):
  job_name = 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  print('Launching Dataflow job {} ... hang on'.format(job_name))
  OUTPUT_DIR = 'gs://{0}/taxifare/taxi_preproc/'.format(BUCKET)

  #dictionary of pipeline options
  options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': PROJECT,
    'runner': RUNNER
  }
  #instantiate PipelineOptions object using options dictionary
  opts = beam.pipeline.PipelineOptions(flags=[], **options)
  #instantantiate Pipeline object using PipelineOptions
  with beam.Pipeline(options=opts) as p:
      for phase in ['train', 'valid']:
        query = create_query(phase, EVERY_N) 
        outfile = os.path.join(OUTPUT_DIR, '{}.csv'.format(phase))
        (
          p | 'read_{}'.format(phase) >> beam.io.Read(beam.io.BigQuerySource(query=query))
            | 'tocsv_{}'.format(phase) >> beam.Map(to_csv)
            | 'write_{}'.format(phase) >> beam.io.Write(beam.io.WriteToText(outfile))
        )
  print("Done")

Run pipeline locally. This takes up to <b>5 minutes</b>.  You will see a message "Done" when it is done.

In [10]:
preprocess(50*10000, 'DirectRunner')

Launching Dataflow job preprocess-taxifeatures-181021-041751 ... hang on


Done


In [16]:
%%bash
gsutil ls -l gs://$BUCKET/taxifare/taxi_preproc/

    108804  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00000-of-00005
    114333  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00001-of-00005
    113893  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00002-of-00005
    114723  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00003-of-00005
    114928  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00004-of-00005
    106755  2018-10-21T04:12:40Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/valid.csv-00000-of-00002
    113586  2018-10-21T04:12:40Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/valid.csv-00001-of-00002
TOTAL: 7 objects, 787022 bytes (768.58 KiB)


## Run Beam pipeline on Cloud Dataflow
Run pipleline on cloud on a larger sample size.

In [24]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/taxi_preproc/
fi

In [ ]:
preprocess(50*100, 'DataflowRunner') 
#change first arg to None to preprocess full dataset

Launching Dataflow job preprocess-taxifeatures-181021-043944 ... hang on


Once the job completes, observe the files created in Google Cloud Storage

In [25]:
%%bash
gsutil ls -l gs://$BUCKET/taxifare/taxi_preproc/

    108804  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00000-of-00005
    114333  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00001-of-00005
    113893  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00002-of-00005
    114723  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00003-of-00005
    114928  2018-10-21T04:12:41Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/train.csv-00004-of-00005
    106755  2018-10-21T04:12:40Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/valid.csv-00000-of-00002
    113586  2018-10-21T04:12:40Z  gs://rostlab-181304-ml/taxifare/taxi_preproc/valid.csv-00001-of-00002
TOTAL: 7 objects, 787022 bytes (768.58 KiB)


In [19]:
%%bash
#print first 10 lines of first shard of train.csv
gsutil cat "gs://$BUCKET/taxifare/taxi_preproc/train.csv-00000-of-*" | head

6.1,Sat,3,-74.005513,40.74069,-73.98978,40.729527,1.0,2010-01-30 03:00:00.000000-74.005540.740740.7295-73.9898
6.1,Thu,3,-74.00006,40.727142,-73.982435,40.72488,1.0,2012-06-14 03:17:00.000000-74.000140.727140.7249-73.9824
6.1,Tue,4,-73.98821,40.764537,-73.978608,40.783388,2.0,2009-11-03 04:46:55.000000-73.988240.764540.7834-73.9786
6.1,Mon,6,-73.964098,40.771065,-73.975158,40.752819,1.0,2012-06-25 06:01:48.000000-73.964140.771140.7528-73.9752
6.1,Sat,6,-74.005195,40.706787,-73.988503,40.71854,3.0,2011-07-09 06:12:00.000000-74.005240.706840.7185-73.9885
6.1,Fri,7,-74.003218,40.727604,-73.976228,40.719043,1.0,2010-01-15 07:30:05.000000-74.003240.727640.719-73.9762
6.1,Wed,7,-73.948706,40.782377,-73.963411,40.774713,1.0,2009-07-01 07:53:10.000000-73.948740.782440.7747-73.9634
6.1,Tue,7,-73.952269,40.766104,-73.969499,40.769078,1.0,2009-03-31 07:51:25.000000-73.952340.766140.7691-73.9695
6.1,Tue,7,-73.992171,40.744027,-73.979134,40.756059,1.0,2012-03-13 07:35:19.000000-73.992240.74440.7561